# Datasheet Comparator - MVP

# 📌 Section A: Setup

In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import fitz  # PyMuPDF for PDF parsing
import pandas as pd

# Load OpenAI API key from environment variable (recommended)

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define paths to datasheets

In [ ]:
pdf_path_1 = "./datasheets/part_old.pdf"
pdf_path_2 = "./datasheets/part_new.pdf"

# 📌 Section B: Extract text from datasheets

In [ ]:
def extract_text_from_pdf(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

text_old = extract_text_from_pdf(pdf_path_1)
text_new = extract_text_from_pdf(pdf_path_2)

# 📌 Section C: Use ChatGPT to summarize and compare

In [ ]:
def summarize_datasheet(text, part_name):
    prompt = f"""
    Summarize the most important technical characteristics of the electronic component '{part_name}' based on this datasheet text:
    ---
    {text}
    ---
    Give a structured list of properties like voltage, current, dimensions, operating temperature, etc.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

summary_old = summarize_datasheet(text_old, "Old Part")
summary_new = summarize_datasheet(text_new, "New Part")


def compare_parts(text1, text2):
    prompt = f"""
    Compare the following two summaries of electronic components and evaluate whether the second part is a valid replacement for the first one.
    Identify any differences in electrical specs, mechanical dimensions, and compliance with medical device requirements.
    Suggest what changes would be required to use the second part in place of the first (e.g., schematic/layout changes).
    
    Old Part Summary:
    {text1}

    New Part Summary:
    {text2}

    Provide a table of differences and a short final recommendation.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

comparison_result = compare_parts(summary_old, summary_new)

# 📌 Section D: Display results

In [ ]:
print("\n=== Summary: Old Part ===\n")
print(summary_old)

print("\n=== Summary: New Part ===\n")
print(summary_new)

print("\n=== Comparison and Recommendation ===\n")
print(comparison_result)

# 📌 Section E: Next Steps (to be developed)

# - Parse key properties into structured tables (e.g., using regex or ChatGPT)

# - Automatically download datasheets from distributor websites

# - Search for compatible parts via web APIs

# - Export results to Excel or Markdown